<a href="https://colab.research.google.com/github/LeeMinJun0102/itwill_middle_project/blob/main/%EA%B0%95%EC%9B%90%EB%8F%841.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box, Point
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from collections import defaultdict
# from streamlit_folium import st_folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 강원도

## 강원도 그리드

In [ ]:
# 1. 강원도만 필터링
gangwon_path = "/content/drive/MyDrive/중간 프로젝트/자료 찾기/전국 행정구역(광역시 단위) api/ctp_rvn.shp"
gdf_all = gpd.read_file(gangwon_path, encoding='cp949').to_crs(epsg=4326)
gangwon = gdf_all[gdf_all['CTP_KOR_NM'] == '강원특별자치도']

# 2. bounding box
minx, miny, maxx, maxy = gangwon.total_bounds
grid_size = 0.05

# 3. 전체 grid 생성
grid_cells = []
grid_ids = []
grid_id = 1

x = minx
while x < maxx:
    y = miny
    while y < maxy:
        grid_cells.append(box(x, y, x + grid_size, y + grid_size))
        grid_ids.append(f'grid_{grid_id}')
        grid_id += 1
        y += grid_size
    x += grid_size

grid = gpd.GeoDataFrame({'grid_id': grid_ids, 'geometry': grid_cells}, crs='EPSG:4326')

# 4. 공간조인으로 강원도와 겹치는 grid만 추출
gangwon_grid = gpd.sjoin(grid, gangwon, how='inner', predicate='intersects').drop(columns='index_right')

In [ ]:
# 1. 강원도만 필터링
gangwon_path = "/content/drive/MyDrive/중간 프로젝트/자료 찾기/전국 행정구역(광역시 단위) api/ctp_rvn.shp"
gdf_all = gpd.read_file(gangwon_path, encoding='cp949').to_crs(epsg=4326)
gangwon = gdf_all[gdf_all['CTP_KOR_NM'] == '강원특별자치도']
gangwon

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,geometry
15,51,Gangwon-do,강원특별자치도,"MULTIPOLYGON (((129.34455 37.1713, 129.34456 3..."


In [ ]:
gangwon_grid

,grid_id,geometry,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM
24,grid_25,"POLYGON ((127.14504 38.22783, 127.14504 38.277...",51,Gangwon-do,강원특별자치도
25,grid_26,"POLYGON ((127.14504 38.27783, 127.14504 38.327...",51,Gangwon-do,강원특별자치도
54,grid_55,"POLYGON ((127.19504 38.12783, 127.19504 38.177...",51,Gangwon-do,강원특별자치도
55,grid_56,"POLYGON ((127.19504 38.17783, 127.19504 38.227...",51,Gangwon-do,강원특별자치도
56,grid_57,"POLYGON ((127.19504 38.22783, 127.19504 38.277...",51,Gangwon-do,강원특별자치도
...,...,...,...,...,...
1412,grid_1413,"POLYGON ((129.34504 37.22783, 129.34504 37.277...",51,Gangwon-do,강원특별자치도
1413,grid_1414,"POLYGON ((129.34504 37.27783, 129.34504 37.327...",51,Gangwon-do,강원특별자치도
1442,grid_1443,"POLYGON ((129.39504 37.12783, 129.39504 37.177...",51,Gangwon-do,강원특별자치도
1443,grid_1444,"POLYGON ((129.39504 37.17783, 129.39504 37.227...",51,Gangwon-do,강원특별자치도


## 로드킬

In [ ]:
file_path = 'https://github.com/LeeMinJun0102/test/raw/refs/heads/main/roadkill'

In [ ]:
rk = pd.read_csv(file_path, sep='\t', names=['number', 'road_kill', 'address', '날짜', '시간', '지역', '경도', '위도'], skiprows=1)
# 마지막 행 제거
# rk = rk[:-1]
# 연도 파생변수 생성
rk['연도'] = rk['날짜'].str.split('-').str[0]

In [ ]:
rk = rk[['경도', '위도', '연도']]

In [ ]:
# 전국 로드킬
rk

,경도,위도,연도
0,127.214314,37.593123,2020
1,127.324362,36.528559,2020
2,127.328889,36.524532,2020
3,127.479598,36.475791,2020
4,127.238288,36.470593,2020
...,...,...,...
9476,126.800164,36.449393,2022
9477,128.904923,37.770272,2022
9478,127.098973,37.308128,2022
9479,127.071588,37.240608,2022


## 강원도 + 로드킬 + 그리드

In [ ]:
# GeoDataFrame 생성
rk_points = gpd.GeoDataFrame(rk,
                              geometry=gpd.points_from_xy(rk['경도'], rk['위도']))

In [ ]:
# 좌표계(CRS) 설정
rk_points.set_crs(epsg=4326, inplace=True)

,경도,위도,연도,geometry
0,127.214314,37.593123,2020,POINT (127.21431 37.59312)
1,127.324362,36.528559,2020,POINT (127.32436 36.52856)
2,127.328889,36.524532,2020,POINT (127.32889 36.52453)
3,127.479598,36.475791,2020,POINT (127.4796 36.47579)
4,127.238288,36.470593,2020,POINT (127.23829 36.47059)
...,...,...,...,...
9476,126.800164,36.449393,2022,POINT (126.80016 36.44939)
9477,128.904923,37.770272,2022,POINT (128.90492 37.77027)
9478,127.098973,37.308128,2022,POINT (127.09897 37.30813)
9479,127.071588,37.240608,2022,POINT (127.07159 37.24061)


In [ ]:
# 강원도 로드킬 = 전국 로드킬(rk_points) + 강원도 좌표(gangwon) -> 조인(sjoin)
gangwon_rk = gpd.sjoin(rk_points, gangwon, how='inner', predicate='within').drop(columns='index_right')


# 강원도 로드킬/그리드 = 강원도 로드킬(gangwon_rk) + 강원도 그리드(gangwon_grid) -> 조인(sjoin)
gangwon_rk_grid = gpd.sjoin(gangwon_rk, gangwon_grid, how='right', predicate='within')
# gangwon_rk_grid = gpd.sjoin(gangwon_grid, gangwon_rk, how='left', predicate='contains')

# 필요한 컬럼만
gangwon_rk_grid = gangwon_rk_grid[['grid_id', '경도', '위도']]

In [ ]:
gangwon_rk_grid['grid_id']

,grid_id
24,grid_25
25,grid_26
54,grid_55
55,grid_56
56,grid_57
...,...
1413,grid_1414
1413,grid_1414
1442,grid_1443
1443,grid_1444


## 생태통로

In [ ]:
eco_road_path = '/content/drive/MyDrive/중간 프로젝트/new/변수/생태통로(수, 이용률) 완/생태통로 시설정보 목록.xlsx'

In [ ]:
eco_road = pd.read_excel(eco_road_path)
eco_road = eco_road[['생태통로 유형', '경도', '위도']]

In [ ]:
# 전국 생태통로
eco_road

,생태통로 유형,경도,위도
0,육교형,127.114921,37.176011
1,육교형,127.120724,37.193110
2,터널형,127.730804,37.033002
3,터널형,126.595165,36.275960
4,터널형,126.596251,36.279848
...,...,...,...
609,육교형,127.064433,37.287872
610,육교형,127.067810,37.280419
611,육교형,127.074036,37.282903
612,육교형,128.505714,38.216200


## 강원도 + 생태통로 + 그리드

In [ ]:
# GeoDataFrame 생성
eco_road_points = gpd.GeoDataFrame(eco_road,
                              geometry=gpd.points_from_xy(eco_road['경도'], eco_road['위도']))

In [ ]:
# 좌표계(CRS) 설정
eco_road_points.set_crs(epsg=4326, inplace=True)

,생태통로 유형,경도,위도,geometry
0,육교형,127.114921,37.176011,POINT (127.11492 37.17601)
1,육교형,127.120724,37.193110,POINT (127.12072 37.19311)
2,터널형,127.730804,37.033002,POINT (127.7308 37.033)
3,터널형,126.595165,36.275960,POINT (126.59516 36.27596)
4,터널형,126.596251,36.279848,POINT (126.59625 36.27985)
...,...,...,...,...
609,육교형,127.064433,37.287872,POINT (127.06443 37.28787)
610,육교형,127.067810,37.280419,POINT (127.06781 37.28042)
611,육교형,127.074036,37.282903,POINT (127.07404 37.2829)
612,육교형,128.505714,38.216200,POINT (128.50571 38.2162)


In [ ]:
# 강원도 생태통로 = 전국 생태통로(eco_road_points) + 강원도 좌표(gangwon) -> 조인(sjoin)
gangwon_eco_road = gpd.sjoin(eco_road_points, gangwon, how='inner', predicate='within').drop(columns='index_right')


# 강원도 생태통로/그리드 = 강원도 생태통로(gangwon_rk) + 강원도 그리드(gangwon_grid) -> 조인(sjoin)
gangwon_eco_road_grid = gpd.sjoin(gangwon_eco_road, gangwon_grid, how='right', predicate='within')
# gangwon_eco_road_grid = gpd.sjoin(gangwon_grid, gangwon_eco_road, how='left', predicate='contains')

# 필요한 컬럼만
gangwon_eco_road_grid = gangwon_eco_road_grid[['grid_id', '경도', '위도']]